# Stack Overflow Analysis



* process data
* what do we want to answer
* features for questions
* train
* evaluation

To begin to understand the data, we wanted to analyze several potential correlations. Notably, we anticipated correlations between:
* length of text and upvote proportion for question/answer
* upvote proportion and upvotes + downvotes for questions/answers
* views and upvote proportion for questions/answers
* question tags and views/response time
* reputation and upvote score for questions/answers

We plotted scatter plots for each pair to visualize the data.

In [ ]:
import matplotlib
# matplotlib.use("svg")
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

# p = plt.scatter(, )
# plt.show()

The scatter plots indicate...

Now that we are familiar with our data, we will try to answer some questions with it. We are interested in determining:
* upvote proportion and upvotes - downvotes
* response time and answerer reputation based on question
* answerer reputation based
* 

* predicting upvotes vs downvotes (proportion? difference? either related to other answers?)
* which factors contribute most? which features are correlated and can be eliminated? we predict that text won't do much
* predicting answerer rep based on other factors
* response time based on question
* correlations:
    * tags and views?
    * length of question and upvotes/downvotes
    * views and upvotes/downvotes
    * upvotes/downvotes and upvotes+downvotes
* 